In [149]:
#!/usr/bin/env python3
# coding: utf-8
import os
import csv
import sklearn
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from functools import reduce
from sklearn.model_selection import train_test_split

In [104]:
data1 = pd.read_csv('eyesClose.csv',header=None)
data2 = pd.read_csv('eyesOpen.csv',header=None)

In [105]:
data1['y'] = 0
data2['y'] = 1

In [106]:
data = pd.concat([data1,data2])

In [107]:
def distance(p1x,p1y,p2x,p2y):
    return (p2x-p1x)**2 + (p2y-p1y)**2

In [108]:
distanceData = pd.DataFrame()
distanceData['y'] = data['y']
distanceData['d0'] = distance(data[0],data[1],data[10],data[11])
distanceData['d1'] = distance(data[0],data[1],data[2],data[3])
distanceData['d2'] = distance(data[2],data[3],data[4],data[5])
distanceData['d3'] = distance(data[4],data[5],data[6],data[7])
distanceData['d4'] = distance(data[6],data[7],data[8],data[9])
distanceData['d5'] = distance(data[8],data[9],data[10],data[11])
distanceData['d6'] = distance(data[2],data[3],data[10],data[11])
distanceData['d7'] = distance(data[4],data[5],data[8],data[9])
distanceData['d8'] = distance(data[0],data[1],data[6],data[7])

distanceData['d0_1'] = distance(data[0+12],data[1+12],data[10+12],data[11+12])
distanceData['d1_1'] = distance(data[0+12],data[1+12],data[2+12],data[3+12])
distanceData['d2_1'] = distance(data[2+12],data[3+12],data[4+12],data[5+12])
distanceData['d3_1'] = distance(data[4+12],data[5+12],data[6+12],data[7+12])
distanceData['d4_1'] = distance(data[6+12],data[7+12],data[8+12],data[9+12])
distanceData['d5_1'] = distance(data[8+12],data[9+12],data[10+12],data[11+12])
distanceData['d6_1'] = distance(data[2+12],data[3+12],data[10+12],data[11+12])
distanceData['d7_1'] = distance(data[4+12],data[5+12],data[8+12],data[9+12])
distanceData['d8_1'] = distance(data[0+12],data[1+12],data[6+12],data[7+12])


In [109]:
distanceData[:2]

,y,d0,d1,d2,d3,d4,d5,d6,d7,d8,d0_1,d1_1,d2_1,d3_1,d4_1,d5_1,d6_1,d7_1,d8_1
0,0,12.324641,9.785758,17.092707,25.213488,20.841498,17.607735,2.310971,2.570773,146.963334,23.537013,22.614329,22.788879,29.926161,28.711934,23.529175,2.437283,2.124413,221.985018
1,0,16.197373,11.782642,17.592918,25.751434,19.964684,18.370951,1.542461,1.683628,159.905142,27.112175,24.830303,23.682798,27.839759,26.352289,23.870057,1.261470,0.992560,227.941520


In [112]:
distanceData_y = distanceData['y']
distanceData=distanceData.drop(['y'],1)

In [114]:
x_train, x_test, y_train, y_test = train_test_split(distanceData, distanceData_y,
                                                    test_size=0.33,
                                                    random_state=0,
                                                    stratify=distanceData_y)


In [174]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)

In [204]:
chosenFeat = ['d6', 'd7','d6_1', 'd7_1']

In [205]:
clf.fit(x_train[chosenFeat], y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [206]:
print(clf.feature_importances_)

[0.27570077 0.19151859 0.29026319 0.24251745]


In [207]:
fl = list(zip(chosenFeat, clf.feature_importances_))

In [208]:
fl2 = sorted(fl, key=lambda x: x[1],reverse=True)
fl2 = list(filter(lambda x: x[1] > 0.1,fl2))
fl2

[('d6_1', 0.29026319438485965),
 ('d6', 0.2757007653071788),
 ('d7_1', 0.2425174479294267),
 ('d7', 0.19151859237853483)]

In [209]:
pred = clf.predict(x_test[chosenFeat])

In [210]:
sklearn.metrics.accuracy_score(y_test,pred)

0.9878787878787879

In [212]:
classifier = clf.fit(distanceData[chosenFeat], distanceData_y)

In [214]:
import pickle
with open('my_dumped_classifier.pkl', 'wb') as fid:
    pickle.dump(classifier, fid)

# with open('my_dumped_classifier.pkl', 'rb') as fid:
#     gnb_loaded = pickle.load(fid)